In [1]:
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
from oneDtoyTask.oneDtoyTask import ToyTask, OneDfield, TwoWayComChannel
from agents.agent_Aria import Aria



In [2]:
epochs = 100
opt_params = {"lr":0.0005, "batch_size":10, "gamma":0.999}
agent0 = Aria(opt_params=opt_params).float()
agent1 = Aria(opt_params=opt_params).float()
np.random.seed(1)
field = OneDfield(speed=10)
Task = ToyTask(field=field, observationMappingFct=lambda x: (x>0.5).astype(int), comChannel=TwoWayComChannel())



In [3]:
obs, downlink_msgs = Task.reset()
losses = []
torch.autograd.set_detect_anomaly(True)
for epoch in range(epochs):
    
    a0, m0u =  agent0.select_action(obs[0], downlink_msgs[0])
    a1, m1u = agent1.select_action(obs[1], downlink_msgs[1])
    m0u_ = np.zeros(4)
    m1u_ = np.zeros(4)
    m0u_[m0u.item()] = 1
    m1u_[m1u.item()] = 1
    (obs_, downlink_msgs_), r, done = Task.step([a0, a1], [m0u_, m1u_])
    loss0 = agent0.train_on_batch([obs[0], downlink_msgs[0]], r[0])
    loss1 = agent1.train_on_batch([obs[1], downlink_msgs[0]], r[1])
    obs = obs_
    downlink_msgs = downlink_msgs_
    if loss0 is not None:
        print("loss agent 0 ", loss0, " loss agent 1 ", loss1)
        losses.append([loss0, loss1])